# 아마존 타이탄을 활용한 작은 파일의 텍스트 요약

> *이 노트북에서 사용하는 베드록 모델에 대한 허용 목록에 포함되어 있지 않다면 오류가 발생할 수 있습니다*

> *이 노트북은 세이지메이커 스튜디오의 **`Data Science 3.0`** 커널에서 잘 작동할 것입니다*


## 개요

이 예시에서는 소량의 데이터(문자열 데이터)를 아마존 베드록 API(아마존 타이탄 모델 사용)에 직접 입력하고 해당 텍스트를 요약하는 인스트럭션을 내릴 것입니다.

### 아키텍처

![](./images/41-text-simple-1.png)

이 아키텍처에서

1. 작은 텍스트 조각(또는 작은 파일)이 로드됩니다.
1. 파운데이션 모델이 해당 데이터를 처리합니다.
1. 모델이 입력된 텍스트의 요약과 함께 응답을 반환합니다.

### 활용 사례

이 접근 방식은 통화 기록, 회의 기록, 책, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

### 과제
이 접근 방식은 입력 텍스트나 파일이 모델 컨텍스트 길이 내에 맞을 때 사용할 수 있습니다. `02.long-text-summarization-titan.ipynb` 노트북에서는 사용자가 토큰 제한을 초과하는 대용량 문서를 가지고 있을 때 이 과제를 해결하는 접근 방식을 탐구할 것입니다.


## 설정

In [2]:
%pip install -U boto3 botocore --force-reinstall --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.3.3 requires pyqt5<5.16, which is not installed.
spyder 5.3.3 requires pyqtwebengine<5.16, which is not installed.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.
jupyterlab 3.4.4 requires jupyter-server~=1.16, but you have jupyter-server 2.7.3 which is incompatible.
jupyterlab-server 2.10.3 requires jupyter-server~=1.4, but you have jupyter-server 2.7.3 which is incompatible.
notebook 6.5.6 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.4.0 which is incompatible.
notebook 6.5.6 requires pyzmq<25,>=17, but you have pyzmq 25.1.1 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.0 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data

#### 이제 Boto3를 사용하여 아마존 베드록 SDK에 대한 연결을 설정해 보겠습니다.

In [3]:
import boto3
import json 

bedrock = boto3.client(service_name="bedrock")
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

## Boto3를 사용한 짧은 텍스트 요약
 
아마존 베드록에 대한 API 요청의 세부 사항을 알아보기 위해, 이 노트북에서는 Boto3 작업을 감싸서 더 간단한 API를 제공하는 langchain에 의존하기보다는 Boto3를 통해 API 요청을 생성하고 전송하는 방법을 소개합니다.

### Boto3의 InvokeModel 요청 구문


파운데이션 모델에 요청을 보내기 위해 `InvokeModel` API를 사용합니다. 다음은 아마존 타이탄 텍스트 라지에 텍스트를 보내는 API 요청의 예시입니다. `textGenerationConfig`의 추론 매개변수는 사용하려는 모델에 따라 다릅니다. 아마존 타이탄 텍스트의 추론 매개변수는 다음과 같습니다.
- **maxTokenCount**는 생성된 응답에서 사용할 최대 토큰 수를 구성합니다. (정수, 기본값 512)
- **stopSequences**는 문장이나 목록의 끝과 같은 원하는 지점에서 모델을 중지시키는 데 사용됩니다. 반환된 응답에는 중지 시퀀스가 포함되지 않습니다.
- **temperature**는 다음 토큰에 대한 확률 밀도 함수를 조절하여 temperature 샘플링 기법을 구현합니다. 이 매개변수를 사용하여 밀도 함수 곡선을 깊게 하거나 평평하게 만들 수 있습니다. 낮은 값은 더 가파른 곡선과 더 결정론적인 응답을 만들고, 높은 값은 더 평평한 곡선과 더 무작위적인 응답을 만듭니다. (부동 소수점, 기본값 0, 최대값 1.5)
- **topP**는 잠재적 선택의 확률을 기반으로 토큰 선택을 제어합니다. Top P를 1.0 미만으로 설정하면 모델은 가장 확률이 높은 옵션만 고려하고 확률이 낮은 옵션은 무시합니다. 결과적으로 더 안정적이고 반복적인 완성이 됩니다.

```python
response = bedrock_runtime.invoke_model(body={
                                   "inputText": "this is where you place your input text",
                                   "textGenerationConfig": {
                                       "maxTokenCount": 4096,
                                       "stopSequences": [],
                                       "temperature":0,
                                       "topP":1
                                       },
                                },
                                modelId="amazon.titan-text-express-v1", 
                                accept=accept, 
                                contentType=contentType)

```

### 요약할 텍스트와 함께 프롬프트 작성하기

이 노트북에서는 토큰 수가 파운데이션 모델의 최대 토큰 수보다 적은 모든 짧은 텍스트를 사용할 수 있습니다. 짧은 텍스트의 예시로, 아마존 베드록 발표에 관한 [AWS 블로그 게시물](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/)의 한 단락을 사용해 보겠습니다.

프롬프트는 `Please provide a summary of the following text.`라는 지시로 시작하고, `<text>` 태그로 둘러싸인 텍스트를 포함합니다.

In [4]:
prompt = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

"""

## 프롬프트와 추론 매개변수로 요청 본문 생성하기

`invoke_model`의 요청 구문에 따라, 위의 프롬프트와 추론 매개변수를 사용하여 요청 본문을 생성합니다.

In [5]:
body = json.dumps({"inputText": prompt, 
                   "textGenerationConfig":{
                       "maxTokenCount":4096,
                       "stopSequences":[],
                       "temperature":0,
                       "topP":1
                   },
                  }) 

## Boto3를 통해 파운데이션 모델 호출하기

여기서는 `modelId`, `accept`, `contentType`과 같은 요청 매개변수를 지정하여 아마존 베드록에 API 요청을 보냅니다. 프롬프트에 따라 아마존 베드록의 파운데이션 모델이 텍스트를 요약합니다.

In [8]:
modelId = 'amazon.titan-text-express-v1' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('results')[0].get('outputText'))

AWS has announced Amazon Bedrock, a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. It will offer the ability to access a range of powerful FMs for text and images through a scalable, reliable, and secure AWS managed service. Customers can easily find the right model for what they're trying to get done, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS tools and capabilities they are familiar with, without having to manage any infrastructure.


위의 예시에서 베드록 서비스는 주어진 프롬프트에 대한 전체 요약을 단일 출력으로 생성합니다. 이는 출력에 많은 양의 토큰이 포함된 경우 느릴 수 있습니다.

아래에서는 베드록을 사용하여 출력을 스트리밍하는 방법을 탐색해 보겠습니다. 이를 통해 사용자는 모델이 생성하는 대로 출력을 소비하기 시작할 수 있습니다. 이를 위해 베드록은 `invoke_model_with_response_stream` API를 지원하며, 이는 출력을 청크 형태로 스트리밍하는 `ResponseStream`을 제공합니다.

In [9]:
response = bedrock_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

[{'chunk': {'bytes': b'{"outputText":"AWS has announced Amazon Bedrock, a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. It will offer the ability to access a range of powerful F","index":0,"totalOutputTextTokenCount":null,"completionReason":null,"inputTextTokenCount":255}'}},
 {'chunk': {'bytes': b'{"outputText":"Ms for text and images through a scalable, reliable, and secure AWS managed service. Customers can easily find the right model for what they\'re trying to get done, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS tools and capabilities they are familiar with, without having to manage any infrastructure.","index":0,"totalOutputTextTokenCount":119,"completionReason":"FINISH","inputTextTokenCount":null,"amazon-bedrock-invocationMetrics":{"inputTokenCount":255,"outputTokenCount":119,"invocationLatency":3573,"firstByteLatency":2162}}'}

전체 출력을 한 번에 생성하는 대신, 베드록은 모델로부터 더 작은 청크를 보냅니다. 이는 사용자가 소비할 수 있는 방식으로 표시될 수 있습니다.

In [10]:
from IPython.display import display_markdown, Markdown, clear_output

In [11]:
response = bedrock_runtime.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

clear_output(wait=True)
print(''.join(output))

AWS has announced Amazon Bedrock, a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. It will offer the ability to access a range of powerful FMs for text and images through a scalable, reliable, and secure AWS managed service. Customers can easily find the right model for what they're trying to get done, get started quickly, privately customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS tools and capabilities they are familiar with, without having to manage any infrastructure.


## 결론
이제 아마존 베드록 API에 대한 기본적인 접근을 제공하는 `boto3` SDK를 사용하는 실험을 해보았습니다. 이 API를 사용하여 아마존 베드록에 관한 AWS 뉴스의 요약을 전체 출력 생성과 스트리밍 출력 생성이라는 두 가지 다른 방식으로 생성하는 사용 사례를 보았습니다.

### 주요 내용
- 이 노트북 수정을 통한 앤트로픽 클로드와 AI21 Labs의 Jurassic 모델과 같은 아마존 베드록의 다양한 모델 실험
- 특정 사용 사례에 맞는 프롬프트 변경 및 다양한 모델 출력 평가
- 토큰 길이 조정을 통한 서비스의 처리 속도와 반응성 이해
- 더 나은 출력을 위한 다양한 프롬프트 엔지니어링 원칙 적용